In [ ]:
!pip install transformers peft torch bitsandbytes accelerate huggingface_hub trl

In [ ]:
from google.colab import userdata
hf_token = userdata.get('hf_token')
!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `All App` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `All App`


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

model_name = "HuggingFaceH4/zephyr-7b-beta"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=getattr(torch, "float16"), bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant= False),
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   100%|          | 8/8 [00:06<00:00, 1.28it/s]

In [ ]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
#model.gradient_checkpointing_enable()
model.gradient_checkpointing_disable()


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="./results",
    save_steps=500,                # Guardar modelo cada 100 pasos, menos frecuente para grandes lotes
    logging_steps=50,              # Mostrar logs cada 50 pasos
    eval_strategy="steps",         # Evaluar cada ciertos pasos
    per_device_train_batch_size=8, # Tamaño de batch mayor para aprovechar GPU
    gradient_accumulation_steps=1, # Ajustado para un batch efectivo razonable
    learning_rate=2e-5,            # Tasa de aprendizaje más baja para mayor estabilidad
    weight_decay=0.01,             # Regularización más fuerte para modelos grandes
    optim="paged_adamw_32bit",     # Optimizador eficiente en GPU
    num_train_epochs=2,            # Incrementa épocas para un ajuste fino más completo
    save_total_limit=3,            # Mantén hasta 3 checkpoints
    fp16=False,                    # Habilita precisión mixta compatible con T4
    bf16=True,                     # No necesario en T4
    max_grad_norm=1.0,             # Límite mayor para gradientes más estables
    max_steps=-1,                  # Ignorado, controlado por num_train_epochs
    warmup_ratio=0.1,              # Warmup más largo para mayor estabilidad inicial
    group_by_length=True,          # Mantén agrupación por longitud para eficiencia en memoria
    lr_scheduler_type="linear",    # Cambia a "linear" para decaimiento suave del learning rate
    report_to="wandb"
)

In [ ]:
from datasets import Dataset, load_dataset, concatenate_datasets
from transformers import DataCollatorForLanguageModeling
import json
import os

# Función para formatear un dataset general
def format_general_dataset(sample):
    """Preserva la estructura del dataset sin agregar redundancias."""
    try:
        system_prompt = sample["<|system|>"]
        user_prompt = sample["<|user|>"]
        assistant_response = sample["<|assistant|>"]

        # Reestructurar para una clave unificada sin alterar el contenido
        sample["text"] = f"<|system|>: {system_prompt}\n<|user|>: {user_prompt}\n<|assistant|>: {assistant_response}"
    except KeyError as e:
        raise ValueError(f"Falta una clave requerida en el dataset: {e}")

    return sample

# Función para formatear un dataset de recomendaciones
def format_recommendations_dataset(sample):
    """Formatea el dataset inicial de recomendaciones."""
    try:
        system_prompt = sample["<|system|>"]
        user_data = json.loads(sample["<|user|>"])["user_profile"]
        assistant_response = sample["<|assistant|>"]

        # Formatear el perfil del usuario en texto claro
        user_profile = (
            f"User Profile:\n"
            f"- Age: {user_data.get('edad')}\n"
            f"- Monthly Income: {user_data.get('ingresos_mensuales')}\n"
            f"- Monthly Savings: {user_data.get('ahorro_mensual')}\n"
            f"- Risk Tolerance: {user_data.get('tolerancia_riesgo')}\n"
            f"- Investment Horizon: {user_data.get('horizonte_inversion')}\n"
            f"- Financial Goal: {user_data.get('objetivo_financiero')}\n"
        )

        # Generar texto formateado para el modelo
        sample["text"] = f"<|system|>: {system_prompt}\n<|user|>: {user_profile}\n<|assistant|>: {assistant_response}"
    except (KeyError, json.JSONDecodeError) as e:
        raise ValueError(f"Error al procesar el dataset `recomendaciones_iniciales`: {e}")

    return sample

# Definición de archivos de datasets
datasets = {
    "comparaciones_esquematico": os.path.join("q&a_comparaciones_esquematico.jsonl"),
    "recomendaciones_iniciales": os.path.join("recomendaciones_iniciales.jsonl"),
    "comparaciones_conversacionales": os.path.join("q&a_comparaciones_conversacionales.jsonl"),
    "conceptos_inversiones": os.path.join("conceptos_inversiones_Q&A.jsonl")
}

# Procesar datasets con el formateo correspondiente
processed_datasets = {}
for name, path in datasets.items():
    dataset = load_dataset("json", data_files=path)["train"]
    if name == "recomendaciones_iniciales":
        formatted_dataset = dataset.map(format_recommendations_dataset, remove_columns=["<|system|>", "<|user|>", "<|assistant|>"])
    else:
        formatted_dataset = dataset.map(format_general_dataset, remove_columns=["<|system|>", "<|user|>", "<|assistant|>"])

    processed_datasets[name] = formatted_dataset

# Combinar datasets
combined_dataset = concatenate_datasets([processed_datasets[name] for name in processed_datasets])

# Dividir en entrenamiento y evaluación
final_dataset = combined_dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = final_dataset["train"]
eval_dataset = final_dataset["test"]

# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # False porque es un modelo causal, no de modelado de lenguaje enmascarado
)

print(f"Conjunto de entrenamiento: {len(train_dataset)} muestras")
print(f"Conjunto de evaluación: {len(eval_dataset)} muestras")


Conjunto de entrenamiento: 2859 muestras
Conjunto de evaluación: 505 muestras


In [ ]:
from trl import SFTTrainer
from accelerate import Accelerator

trainer = SFTTrainer(model = model,
                    args = training_args,
                    train_dataset = train_dataset,
                    eval_dataset = eval_dataset,
                    peft_config=lora_config,
                    processing_class = tokenizer,
                    data_collator=data_collator,)

In [ ]:
import wandb
wandb_token = userdata.get('wandb_token')
wandb.login(key = wandb_token)
run = wandb.init(
    project='Fine tuning Zephyr 7B',
    job_type="training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ebayego (ebayego-universitat-oberta-de-catalunya). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,1.635000,1.496082
100,1.286700,0.987415
150,0.734800,0.517745
200,0.424300,0.370512
250,0.312300,0.276465
300,0.258800,0.239283
350,0.231300,0.216503
400,0.170100,0.200281
450,0.182800,0.193877
500,0.190400,0.186046


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=716, training_loss=0.43346010439888727, metrics={'train_runtime': 2388.8506, 'train_samples_per_second': 2.394, 'train_steps_per_second': 0.3, 'total_flos': 1.9163662773178368e+17, 'train_loss': 0.43346010439888727, 'epoch': 2.0})

In [ ]:
new_model = "TFG"
trainer.model.save_pretrained(new_model)
wandb.finish()

trainer.model.push_to_hub(new_model, use_temp_dir=False)

print("Fine-tuning completado. Modelo y tokenizador guardados en './final_model'.")

eval/loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁
eval/runtime,▆▇▃▃▆▇▇█▄▇▁▄▄▅
eval/samples_per_second,▃▂▆▆▃▂▂▁▅▂█▅▅▄
eval/steps_per_second,▃▂▆▆▂▂▂▁▅▂█▅▅▄
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/grad_norm,▄▄▅▄▄▄█▁▂▂▂▂▂▂
train/learning_rate,▆█▇▇▆▆▅▄▄▃▃▂▂▁
train/loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁
eval/loss,0.17895
eval/runtime,47.5255


adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

Fine-tuning completado. Modelo y tokenizador guardados en './final_model'.


In [ ]:
model = model.merge_and_unload()

# Guardar el modelo completo
output_dir = "./final_full_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Modelo completo guardado en '{output_dir}'.")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:355: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Modelo completo guardado en './final_full_model'.


In [ ]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1000,  # Límite máximo razonable
    min_length=50,   # Longitud mínima para evitar respuestas truncadas
    temperature=0.9, # Control de aleatoriedad
    top_p=0.95,      # Diversidad de respuestas
    eos_token_id=tokenizer.eos_token_id  # Detenerse al encontrar <eos>
)

Device set to use cuda:0


In [ ]:
prompt = "¿Cuál es la diferencia entre ETFs de acumulación y distribución?"
result = pipe(prompt)
result_text = result[0]['generated_text']
cleaned_text = result_text.split("<|assistant|>")[-1].strip()
print(cleaned_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
`use_cache=True` is in

Las ETFs (Exchange Traded Funds) se pueden clasificar en dos tipos principales: ETFs de acumulación y ETFs de distribución. La principal diferencia entre ambos es cómo se distribuyen los ingresos y los gastos entre los accionistas.

En los ETFs de acumulación, los ingresos y los gastos se acumulan en la cartera de la ETF, y se distribuyen a los accionistas en forma de capital de distribución al final del año fiscal. Esto significa que los accionistas pueden optar por reinvertir los ingresos en la ETF o retirirlos en forma de dividendos. Los ETFs de acumulación pueden ser atractivos para los inversores que buscan un rendimiento más estable y predictible, ya que los ingresos se acumulan y se distribuyen en forma de capital de distribución, lo que puede reducir la volatilidad de los ingresos.

En los ETFs de distribución, los ingresos y los gastos se distribuyen a los accionistas en forma de dividendos mensuales o trimestrales. Esto significa que los accionistas pueden optar por reinverti

In [ ]:
prompt = "¿Cuales son las principales diferencias entre el MSCI World y el SP&500?"
result = pipe(prompt)
print(result[0]['generated_text'].replace("<|assistant|>", "").strip())

¿Cuales son las principales diferencias entre el MSCI World y el SP&500? 

Aunque ambos índices son medidas de rendimiento de la bolsa, hay algunas principales diferencias entre el MSCI World y el SP&500:

1. Composición: El MSCI World es un índice global que mide el desempeño de las principales empresas de 23 países desarrollados y emergentes, mientras que el SP&500 se centra en las empresas más grandes y líderes de la economía estadounidense.

2. Países: El MSCI World incluye empresas de países como Alemania, Japón, Reino Unido, Francia, China, Hong Kong, Taiwán, Sudáfrica, Australia, Nueva Zelanda, Singapur, Taiwán, Corea del Sur, y otros, además de los Estados Unidos. El SP&500 solo incluye empresas de los Estados Unidos.

3. Sectorial: El MSCI World tiene una distribución sectorial más diversa que el SP&500, con una mayor representación de empresas de la industria tecnológica, de la salud y de la energía renovable. El SP&500 tiene una mayor representación de empresas de la industr

In [ ]:
prompt = ("<|system|>: Eres un asistente financiero personal especializado en inversiones. "
          "Tu objetivo es ayudar a los usuarios con cualquier tema relacionado con inversiones, finanzas o economía. "
          "Para cada respuesta:\n"
          "1. Primero, busca los datos adecuados de tus conocimientos aprendidos o de las bases de datos externas disponibles.\n"
          "2. Luego, analiza cuidadosamente la consulta para dar una respuesta clara, precisa y personalizada.\n"
          "3. Mantente amable, profesional y enfocado únicamente en temas relacionados con inversiones, finanzas y economía.\n"
          "<|user|>: ¿En qué aspectos son diferentes UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C?"
          "<|assistant|>:"
)
result = pipe(prompt)
print(result[0]['generated_text'])

<|system|>: Eres un asistente financiero personal especializado en inversiones. Tu objetivo es ayudar a los usuarios con cualquier tema relacionado con inversiones, finanzas o economía. Para cada respuesta:
1. Primero, busca los datos adecuados de tus conocimientos aprendidos o de las bases de datos externas disponibles.
2. Luego, analiza cuidadosamente la consulta para dar una respuesta clara, precisa y personalizada.
3. Mantente amable, profesional y enfocado únicamente en temas relacionados con inversiones, finanzas y economía.<|user|>: ¿En qué aspectos son diferentes UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C?<|assistant|>: 
<|assistant|>
Las UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (EUR) A-dis y Xtrackers Euro Stoxx 50 UCITS ETF 1C son dos ETFs (Exchange Traded Funds) que se comercializan en Europa, pero presentan diferencias en sus objetivos de inversión.

La UBS ETF - Factor MSCI EMU Low Volatility UCITS ETF (E

In [ ]:
!zip -r /content/file.zip /content
from google.colab import files
files.download("/content/file.zip")

updating: content/ (stored 0%)
updating: content/.config/ (stored 0%)
updating: content/.config/.last_opt_in_prompt.yaml (stored 0%)
updating: content/.config/default_configs.db (deflated 98%)
updating: content/.config/.last_survey_prompt.yaml (stored 0%)
updating: content/.config/gce (stored 0%)
updating: content/.config/config_sentinel (stored 0%)
updating: content/.config/.last_update_check.json (deflated 22%)
updating: content/.config/logs/ (stored 0%)
updating: content/.config/logs/2024.12.19/ (stored 0%)
updating: content/.config/logs/2024.12.19/14.20.18.151587.log (deflated 58%)
updating: content/.config/logs/2024.12.19/14.20.29.520330.log (deflated 57%)
updating: content/.config/logs/2024.12.19/14.20.16.940511.log (deflated 87%)
updating: content/.config/logs/2024.12.19/14.19.43.316528.log (deflated 93%)
updating: content/.config/logs/2024.12.19/14.20.30.129972.log (deflated 57%)
updating: content/.config/logs/2024.12.19/14.20.05.781718.log (deflated 58%)
updating: content/.con

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>